In [ ]:
!pip install ../input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/ > /dev/null

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import sys

import numpy as np
import pandas as pd
import torchvision #package consists of popular datasets, model architectures, and common image transformations for computer vision.
import torch.nn as nn #neural network model
from tqdm import tqdm #progress bars to Python code 
from PIL import Image, ImageFile #adds support for opening, manipulating, and saving many different image file formats.
from torch.utils.data import Dataset #An abstract class representing a Dataset.
import torch #pytorch
from torchvision import transforms #image tranformation 
import os #functions for interacting with the operating system.
import pretrainedmodels 
from efficientnet_pytorch import EfficientNet 
device = torch.device("cuda:0") # used to send the model to selected GPU
ImageFile.LOAD_TRUNCATED_IMAGES = True
# Any results you write to the current directory are saved as output.

In [ ]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Rotate,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    #MultiplicativeNoise,
    #CoarseDropout,
    CLAHE,
    RandomBrightnessContrast,    
    Cutout,
    RandomGamma,
    Rotate,
    ShiftScaleRotate ,
    GaussNoise,
    Blur,
    #GlassBlur,
    Normalize,
    MotionBlur,
    MedianBlur,   
    IAAPiecewiseAffine,
    GaussianBlur
)

In [ ]:
class RetinopathyDatasetTest(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)  #returns the number of items in an object.

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/test_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = image.resize((224, 224), resample=Image.BILINEAR)
        image = np.asarray( image, dtype=np.uint8 ) #image is converted into an array datatype integer
        if self.transform:
            result = self.transform(image=image)
            image = result['image']
        return {'image': transforms.ToTensor()(image)} #Data loader need the input in the form of tensor


In [ ]:
import torch.nn as nn
import torch.nn.functional as F  #(uncomment if needed,but you likely already have it)

#Mish - "Mish: A Self Regularized Non-Monotonic Neural Activation Function"
#https://arxiv.org/abs/1908.08681v1
#implemented for PyTorch / FastAI by lessw2020 
#github: https://github.com/lessw2020/mish

class Mish(nn.Module):
    def __init__(self):
        super().__init__() #the function returns a temporary object that allows reference to a parent class 

    def forward(self, x):
        #inlining this saves 1 second per epoch (V100 GPU) vs having a temp x and then returning x(!)
        return x *( torch.tanh(F.softplus(x))) #provides support for the hyperbolic tangent function in PyTorch.

In [ ]:
class AdaptiveConcatPool2d(nn.Module):   
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`."
    def __init__(self, sz):
      super().__init__()
      "Output will be 2*sz or 2 if sz is None"
      self.output_size = sz or 1
      self.ap = nn.AdaptiveAvgPool2d(self.output_size)
      self.mp = nn.AdaptiveMaxPool2d(self.output_size)

    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
    
    #uses adaptive average pooling and adaptive max pooling and concatenates them both.
    #Max pooling extracts the most important features like edges whereas, average pooling extracts features so smoothly.

In [ ]:
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False): 
      super().__init__()
      self.full = full
    def forward(self, x): return x.view(-1) if self.full else x.view(x.size(0), -1)
    
    #used to get a copy of an given array collapsed into one dimension.

In [ ]:
from torch.nn.parameter import Parameter

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
def convert_to_gem(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.AdaptiveAvgPool2d):
            setattr(model, child_name, GeM())
        else:
            convert_to_gem(child)

In [ ]:
def bn_drop_lin( n_in:int, n_out:int, actn:nn.Module,bn:bool=True, p:float=0.):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

#Since we are computing the mean and standard deviation from a single batch as opposed to computing it from the entire data. 
#Batch normalization is done individually at each hidden neuron in the network.
#concept of dropout in neural networks specifically how it helps to reduce overfitting and generalization error.
#Dropout is a regularization technique that “drops out” or “deactivates” few neurons in the neural network randomly in order to avoid the problem of overfitting.
#Each Linear Module computes output from input using a linear function, and holds internal Tensors for its weight and bias.


In [ ]:
class Head(nn.Module):
    def __init__(self, nc, n, ps=0.5):
        super(Head,self).__init__()
        layers = [AdaptiveConcatPool2d(1), Mish(), Flatten()] + \
            bn_drop_lin(nc*2, 512, Mish(),True, ps) + \
            bn_drop_lin(512, n, None, True, ps)
        self.fc = nn.Sequential(*layers)
        self._init_weight()
        
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
        
    def forward(self, x):
        return self.fc(x)

In [ ]:
arch1 = pretrainedmodels.__dict__['se_resnext50_32x4d']
arch2 = 'efficientnet-b2'

In [ ]:
class DRModel1(nn.Module):
    def __init__(self, arch1 = arch1 , pre=True):
        super(DRModel1,self).__init__()
        m = arch1(pretrained='imagenet') if pre else arch1(pretrained=None) #base model inherit
        conv = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) 
        #w = (m.layer0.conv1.weight.sum(1)).unsqueeze(1)
        #conv.weight = nn.Parameter(w)
        self.layer0 = nn.Sequential(conv, m.layer0.bn1, m.layer0.relu1, m.layer0.pool)
        self.layer1 = m.layer1
        self.layer2 = m.layer2
        self.layer3 = m.layer3
        self.layer4 = nn.Sequential(m.layer4[0], m.layer4[1], m.layer4[2]) #

        
        nc = self.layer4[-1].se_module.fc2.out_channels       # changes as per architecture
        self.head = Head(nc,1)
#         self.head1 = Head(nc,n[0])
#         self.base_model = m
#         self.head = Head(1000,1)
#         convert_sigmoid_to_mish(self.base_model)
#         convert_relu_to_mish(self.base_model)
        
        
    def forward(self, x):    
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.head(x)
        
        return x

In [ ]:
class DRModel2(nn.Module):
    def __init__(self, arch2 = arch2 , pre=True):
        super(DRModel2,self).__init__()
        m = EfficientNet.from_pretrained(arch2 ) if pre else EfficientNet.from_name(arch2)
        conv = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        #w = (m.layer0.conv1.weight.sum(1)).unsqueeze(1)
        #conv.weight = nn.Parameter(w)
        #self.conv1=conv
        self.base = m
        self.base._fc = nn.Linear(1408,1,bias = True)
        
        #nc = self.layer4[-1].se_module.fc2.out_channels       # changes as per architecture
        #self.head = Head(nc,1)
#         self.head1 = Head(nc,n[0])
#         self.base_model = m
#         self.head = Head(1000,1)
#         convert_sigmoid_to_mish(self.base_model)
#         convert_relu_to_mish(self.base_model)
        
        
    def forward(self, x):    
        #x= self.conv1(x)
        x = self.base(x)
        return x

In [ ]:
model1 = DRModel1(pre=False)
#convert_to_gem(model1)
model1 = model1.to(device)
model2 = DRModel2(pre=False)
convert_to_gem(model2)
model2 = model2.to(device)

In [ ]:
model1.load_state_dict(torch.load("../input/modelweights/model_S.pth"))
model2.load_state_dict(torch.load("../input/updatedmodelweights/model_Enew.pth"))

In [ ]:
model1.eval()
model2.eval()

In [ ]:
test_transforms = Compose([ 
    ShiftScaleRotate(p=1,rotate_limit=180, shift_limit=0.2,scale_limit=0.2),
    Normalize(mean=(0.0692),std=(0.2051)),
    #Flip(p=0.6),
    GaussianBlur(blur_limit=5)])

In [ ]:
test_dataset = RetinopathyDatasetTest(csv_file='../input/aptos2019-blindness-detection/sample_submission.csv',
                                      transform=test_transforms)

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
preds1 = np.zeros((len(test_dataset), 1))
tk0 = tqdm(test_data_loader,total=int(len(test_data_loader)))
for i, x_batch in enumerate(tk0):
    x_batch = x_batch["image"]
    pred = model1(x_batch.to(device))
    preds1[i * 32:(i + 1) * 32] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [ ]:
preds1

In [ ]:
preds2 = np.zeros((len(test_dataset), 1))
tk1 = tqdm(test_data_loader , total=int(len(test_data_loader)))
for i, x_batch in enumerate(tk1):
    x_batch = x_batch["image"]
    pred = model2(x_batch.to(device))
    preds2[i * 32:(i + 1) * 32] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [ ]:
preds2

In [ ]:
test_preds = (preds1 +preds2) / 2.0
#test_preds = preds2
#test_preds = preds1

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv(r'C:\Users\Admin\Desktop\helpme.csv', index=False)


In [ ]:
i=0
j=0
for i in range(sample.shape[0]):
    if sample.diagnosis[i] == 4:
        j = j+1
        
print (j)
i

In [ ]:
sample1 = pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
#sample1.shape[0]
#sample1.diagnosis = test_preds.astype(int)
#sample1.to_csv(r'C:\Users\Admin\Desktop\fafa2.csv', index=False)
i=0
j=0
for i in range(sample1.shape[0]):
    if sample1.diagnosis[i] == 4:
        j = j+1
print(j)
print(j/sample1.shape[0])

In [ ]:
sample.iloc[1435:1900,:]